# Monthly Reports

## Head FLEX Days Used - Last 12 months

In [109]:
# start with imports
import pandas as pd
import db_functions as dbfnc
import config as cfg
from datetime import date

In [110]:
# grab data from the db and input it to a df
# the query is HeadworkedTable joined with ShowTable joined with HeadNamesTable filtered on ShowName 'FLEX DAY'
sql_file=open(f'{cfg.sql_dir}\head_toil_days.sql')
query = sql_file.read()

with dbfnc.connection(cfg.my_driver, cfg.my_server, cfg.my_db) as conn:
    df = pd.read_sql(query, conn)

In [111]:
# filter for last 12 months
value_to_check = pd.Timestamp(date.today().year-1, date.today().month, date.today().day)
filter_mask = df['ShiftDate'] > value_to_check
df = df[filter_mask]

In [112]:
# create df with FLEX DAYS only with a filter
filter_mask = df['ShowTitle'] == 'FLEX DAY'
df = df[filter_mask]

In [113]:
# turn the datetime64 object into a string object for manipultaion
df['Date_String'] = df['ShiftDate'].dt.strftime('%Y-%m-%d')

In [114]:
# create the fiscal info from the new string object
df['Fiscal Year'] = df.Date_String.apply(dbfnc.fiscal_yr)
df['Fiscal Quarter'] = df.Date_String.apply(dbfnc.fiscal_q)

In [115]:
# sort for presentation
df.sort_values(by='FirstName', inplace=True)
sort_list = ['Fiscal Year', 'Fiscal Quarter']
df.sort_values(by=sort_list, ascending=False, inplace=True)

In [116]:
#create the new df removing uneeded column data
df = df[['Fiscal Year', 'Fiscal Quarter', 'FirstName', 'ShiftDate']].copy()

In [117]:
#.. and style for preentation
df.style.set_properties(**{'text-align':'center'}).hide_index()

Fiscal Year,Fiscal Quarter,FirstName,ShiftDate
2020-2021,2,Arran,2021-02-05 00:00:00
2020-2021,2,David,2021-01-29 00:00:00
2020-2021,2,Jack,2021-01-14 00:00:00
2020-2021,2,Jaclyn,2020-12-11 00:00:00
2020-2021,2,Kris,2021-02-19 00:00:00
2020-2021,2,Matt,2021-02-26 00:00:00
2020-2021,2,Rikki,2021-02-25 00:00:00
2020-2021,2,Terry,2021-02-17 00:00:00
2020-2021,1,Arran,2020-10-16 00:00:00
2020-2021,1,David,2020-11-20 00:00:00


---

## TOIL Payouts to Heads - Current Fiscal Year

In [118]:
import pandas as pd
import db_functions as dbfnc
import config as cfg
from datetime import date

In [119]:
# grab data from the db and input it to a df
# the query is HeadworkedTable joined with ShowTable joined with HeadNamesTable filtered on ShowName 'FLEX DAY'
sql_file=open(f'{cfg.sql_dir}\TOIL_payouts.sql')
query = sql_file.read()

with dbfnc.connection(cfg.my_driver, cfg.my_server, cfg.my_db) as conn:
    df = pd.read_sql(query, conn)

In [120]:
# turn the datetime64 object into a string object for manipultaion
df['Date_String'] = df['PayoutDate'].dt.strftime('%Y-%m-%d')

In [121]:
# filter for current fiscal year only
filter_mask = df.Date_String.apply(dbfnc.current_fiscal) 
df = df[filter_mask]

In [122]:
# sum and group each heads hours. 

In [123]:
# create the new df removing uneeded column data
df = df[['FirstName', 'PayoutinHrs']].copy()

In [124]:
#.. and style for preentation
df.style.set_properties(**{'text-align':'center'}).hide_index()

FirstName,PayoutinHrs
Arran,0.250000
Jaclyn,0.400000


---

# Head Salaried Hours Worked - Current Year

In [125]:
import pandas as pd
import db_functions as dbfnc
import config as cfg
from datetime import date

In [126]:
# grab data from the db and input it to a df
# the query is HeadworkedTable joined with ShowTable joined with HeadNamesTable filtered on ShowName 'FLEX DAY'
sql_file=open(f'{cfg.sql_dir}\head_hours_worked.sql')
query = sql_file.read()

with dbfnc.connection(cfg.my_driver, cfg.my_server, cfg.my_db) as conn:
    df = pd.read_sql(query, conn)

In [127]:
# turn the datetime64 object into a string object for manipultaion
df['Date_String'] = df['ShiftDate'].dt.strftime('%Y-%m-%d')

In [128]:
# filter for current fiscal year only
filter_mask = df.Date_String.apply(dbfnc.current_fiscal) 
df = df[filter_mask]

In [129]:
# Overtime conversions
df['OT_convert'] = df.OverTimeHours.apply(lambda x:x*1.5)
df['DT_convert'] = df.DoubleTimeHours.apply(lambda x:x*2)

In [130]:
# Combine all hours
sum_cols =['RegTimeHours', 'OT_convert', 'DT_convert']
df['all_hrs'] = df[sum_cols].sum(axis=1)

In [131]:
#create percent column
df['percent'] = (df['all_hrs'] / df['all_hrs'].sum()) * 100
df.percent = df.percent.round(2)

In [132]:
# create the new df removing uneeded column data
df = df[['AccountCodeID', 'Note', 'all_hrs', 'percent']].copy()

In [133]:
# sum and present the data
df.groupby(['AccountCodeID', 'Note']).sum()

,,all_hrs,percent
AccountCodeID,Note,,
6210-50-504,Jack Singer Salaried Billable,67.50,0.82
7290-50-504,LX Maintenance,61.50,0.76
7420-20-204,Snd Res Co,1.50,0.02
7430-50-504,Sound Maintenance,152.50,1.86
7470-20-204,Stg Res Co,10.50,0.12
7480-50-504,Stage Maintenance,675.75,8.22
8430-50-504,Salaried Admin,3677.00,45.09
8440-20-203,Lance Labour,9.00,0.10
8455-50-504,Vacation/Holiday/Flex/Sick,1011.00,12.60
